In [1]:
using DifferentialEquations
using ParameterizedFunctions
using Sundials
using DataFrames
using CSV

In [2]:
function temp_gen(hour, temp_ref)
    temp_ref + hour / (20 * 24 * 365) + 10 * sin((2 * pi / 24) * hour) + 10 * sin((2 * pi / (24 * 365)) * hour)
end

function I_S(hour)
    0.001 + 0.0005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous SOC input
end

function I_D(hour)
    0.0001 + 0.00005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous DOC input
end

function arrhenius_temp_dep(parameter, temp, Ea, temp_ref)
    decayed_parameter = parameter * exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref))
end

function linear_temp_dep(parameter, temp, Q, temp_ref)
    modified_parameter = parameter - Q * (temp - temp_ref)
end

function analytical_steady_state_AWB_original(SOC_input, DOC_input, u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L, temp_ref)
    u_Q = linear_temp_dep(u_Q_ref, temp_ref, Q, temp_ref)
    V_D = arrhenius_temp_dep(V_D_ref, temp_ref, Ea_V_D, temp_ref)
    V_U = arrhenius_temp_dep(V_U_ref, temp_ref, Ea_V_U, temp_ref)    
    D₀ = -((K_U * (r_E + r_M)) / (r_E + r_M - u_Q * V_U))
    S₀ = -((K_D * r_L * (SOC_input * r_E * (u_Q - 1) - a_MSA * DOC_input * r_M * u_Q + SOC_input * r_M * (-1 + u_Q - a_MSA * u_Q))) / (DOC_input * u_Q * (-a_MSA * r_L * r_M + r_E * V_D) + SOC_input * (r_E * r_L * (u_Q - 1) + r_L * r_M * (-1 + u_Q - a_MSA * u_Q) + r_E * u_Q * V_D)))
    M₀ = -((u_Q * (SOC_input + DOC_input)) / ((r_E + r_M) * (u_Q - 1)))
    E₀ = -((r_E * u_Q * (SOC_input + DOC_input)) / (r_L * (r_E + r_M) * (u_Q - 1))) 
    return [S₀, D₀, M₀, E₀]
end

analytical_steady_state_AWB_original (generic function with 1 method)

In [3]:
########################
###Non-param Constants##
########################

temp_ref = 283

#########################
##ODE System Parameters##
#########################

u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Separate parameters for full ECA
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50

50

In [4]:
###############
##ODE Solving##
###############

tspan = (0., 100000.) #in hours
C₀ = analytical_steady_state_AWB_original(I_S(0), I_D(0), u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L, temp_ref)
print(C₀)
p = [u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L]

function AWB!(du, u, p, t)
    S, D, M, E = u
    u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L = p
    du[1] = dS = I_S(t) + a_MSA * r_M * M - ((arrhenius_temp_dep(V_D_ref, temp_gen(t, temp_ref), Ea_V_D, temp_ref) * E * S) / (K_D + S))
    du[2] = dD = I_D(t) + (1 - a_MSA) * r_M * M + ((arrhenius_temp_dep(V_D_ref, temp_gen(t, temp_ref), Ea_V_D, temp_ref) * E * S) / (K_D + S)) + r_L * E - ((arrhenius_temp_dep(V_U_ref, temp_gen(t, temp_ref), Ea_V_U, temp_ref) * M * D) / (K_U + D))
    du[3] = dM = linear_temp_dep(u_Q_ref, temp_gen(t, temp_ref), Q, temp_ref) * ((arrhenius_temp_dep(V_U_ref, temp_gen(t, temp_ref), Ea_V_U, temp_ref) * M * D) / (K_U + D)) - r_M * M - r_E * M
    du[4] = dE = r_E * M - r_L * E
end

AWB_prob = ODEProblem(AWB!, C₀, tspan, p)
AWB_sol = solve(AWB_prob, CVODE_BDF(), dt = 0.05, saveat = 0:2:100000) #sample result every 2 hour(s), forcing dt = 0.05 hours

[53.60230547550432, 0.11420612813370475, 0.6707317073170732, 0.013414634146341465]

retcode: Success
Interpolation: 1st order linear
t: 50001-element Vector{Float64}:
      0.0
      2.0
      4.0
      6.0
      8.0
     10.0
     12.0
     14.0
     16.0
     18.0
     20.0
     22.0
     24.0
      ⋮
  99978.0
  99980.0
  99982.0
  99984.0
  99986.0
  99988.0
  99990.0
  99992.0
  99994.0
  99996.0
  99998.0
 100000.0
u: 50001-element Vector{Vector{Float64}}:
 [53.60230547550432, 0.11420612813370475, 0.6707317073170732, 0.013414634146341465]
 [53.60110112077525, 0.11451744294844479, 0.6708820294018831, 0.013414636186660986]
 [53.59840728020702, 0.11532410933540727, 0.6711926923078062, 0.01341464265251501]
 [53.5944760308531, 0.11655930969112097, 0.671631202527707, 0.013414656572205762]
 [53.59064084235692, 0.11770005203467468, 0.6720712476406989, 0.013414679052891468]
 [53.5881335801451, 0.11832133007587438, 0.672385198919786, 0.013414708804701619]
 [53.58727533158442, 0.11845731014909311, 0.6725085355579893, 0.01341474256435054]
 [53.58751080726127, 0.118441771267

In [5]:
##########################
##Debugging and Checking##
##########################

#Test if function sticks to steady state with no temperature forcing

function AWB_ss!(du, u, p, t)
    S, D, M, E = u
    u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L = p
    du[1] = dS = I_S(t) + a_MSA * r_M * M - (V_D_ref * E * S) / (K_D + S)
    du[2] = dD = I_D(t) + (1 - a_MSA) * r_M * M + (V_D_ref * E * S) / (K_D + S) + r_L * E - (V_U_ref * M * D) / (K_U + D)
    du[3] = dM = u_Q_ref * (V_U_ref * M * D) / (K_U + D) - r_M * M - r_E * M
    du[4] = dE = r_E * M - r_L * E
end

AWB_ss_prob = ODEProblem(AWB_ss!, C₀, tspan, p)
AWB_ss_sol = solve(AWB_ss_prob, dt = 0.05, saveat = 0:10:100000) #sample result every 10 hours, forcing dt = 0.05 hours)

function AWB_ss2!(du, u, p, t)
    S, D, M, E = u
    u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L = p
    du[1] = dS = I_S(t) + a_MSA * r_M * M - ((arrhenius_temp_dep(V_D_ref, temp_ref, Ea_V_D, temp_ref) * E * S) / (K_D + S))
    du[2] = dD = I_D(t) + (1 - a_MSA) * r_M * M + ((arrhenius_temp_dep(V_D_ref, temp_ref, Ea_V_D, temp_ref) * E * S) / (K_D + S)) + r_L * E - ((arrhenius_temp_dep(V_U_ref, temp_ref, Ea_V_U, temp_ref) * M * D) / (K_U + D))
    du[3] = dM = linear_temp_dep(u_Q_ref, temp_ref, Q, temp_ref) * ((arrhenius_temp_dep(V_U_ref, temp_ref, Ea_V_U, temp_ref) * M * D) / (K_U + D)) - r_M * M - r_E * M
    du[4] = dE = r_E * M - r_L * E
end

AWB_ss2_prob = ODEProblem(AWB_ss2!, C₀, tspan, p)
AWB_ss2_sol = solve(AWB_ss2_prob, dt = 0.05, saveat = 0:10:100000) #sample result every 10 hours, forcing dt = 0.05 hours)

retcode: Success
Interpolation: 1st order linear
t: 10001-element Vector{Float64}:
      0.0
     10.0
     20.0
     30.0
     40.0
     50.0
     60.0
     70.0
     80.0
     90.0
    100.0
    110.0
    120.0
      ⋮
  99890.0
  99900.0
  99910.0
  99920.0
  99930.0
  99940.0
  99950.0
  99960.0
  99970.0
  99980.0
  99990.0
 100000.0
u: 10001-element Vector{Vector{Float64}}:
 [53.60230547550432, 0.11420612813370475, 0.6707317073170732, 0.013414634146341465]
 [53.60232340589836, 0.11420785877830991, 0.670731720007776, 0.013414634146546447]
 [53.6023771922384, 0.11421281471546423, 0.6707318059263293, 0.013414634150733944]
 [53.6024668260002, 0.11422067259270333, 0.6707320304637636, 0.013414634170270714]
 [53.60259229690947, 0.11423113753121587, 0.670732453078428, 0.013414634221939522]
 [53.602753592933496, 0.11424394162652586, 0.6707331276074239, 0.013414634327625012]
 [53.60295070023834, 0.11425883602955951, 0.6707341039114435, 0.013414634512656313]
 [53.60318360320234, 0.114275593

In [6]:
##################
##Exporting Data##
##################

df_AWB_soln = DataFrame(AWB_sol.u, :auto)
df_AWB_soln

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,53.6023,53.6011,53.5984,53.5945,53.5906,53.5881,53.5873,53.5875
2,0.114206,0.114517,0.115324,0.116559,0.1177,0.118321,0.118457,0.118442
3,0.670732,0.670882,0.671193,0.671631,0.672071,0.672385,0.672509,0.672467
4,0.0134146,0.0134146,0.0134146,0.0134147,0.0134147,0.0134147,0.0134147,0.0134148


In [7]:
CSV.write("AWB_synthetic_soln.csv", df_AWB_soln)

"AWB_synthetic_soln.csv"